In [1]:
import pandas as pd
import numpy as np
import json as js

In [2]:
temp_quarter_df = pd.read_csv('../SourceMaterial/Measures/Results/water_result.csv')
conductivity_quarter_df = pd.read_csv('../SourceMaterial/Measures/Results/conductivity_result.csv')
ph_quarter_df = pd.read_csv('../SourceMaterial/Measures/Results/ph_result.csv')

df_list = [temp_quarter_df, conductivity_quarter_df, ph_quarter_df]
name_list = ['temp', 'conductivity', 'ph']
quarter_list = df_list[0]['q'].unique()

In [3]:
temp_quarter_df.head()

,Unnamed: 0,huc,quarter,q,year,var,stdev,avg,max,min,max_time_diff,max_date,min_date
0,0,Totuskey Creek,Q1 2012,Q1,2012,41.227788,6.420887,15.521739,25.0,2.5,37,2019-12-19,2012-01-05
1,1,Totuskey Creek,Q1 2013,Q1,2013,66.830665,8.175002,14.190217,30.0,4.5,37,2019-12-19,2012-01-05
2,2,Totuskey Creek,Q1 2014,Q1,2014,76.105240,8.723832,15.222826,29.5,1.5,37,2019-12-19,2012-01-05
3,3,Totuskey Creek,Q1 2015,Q1,2015,70.460000,8.394046,17.033333,32.0,3.0,37,2019-12-19,2012-01-05
4,4,Totuskey Creek,Q1 2016,Q1,2016,59.024818,7.682761,16.911236,31.5,3.0,37,2019-12-19,2012-01-05


In [4]:
conductivity_quarter_df.head()

,Unnamed: 0,huc,quarter,q,year,var,stdev,avg,max,min,max_time_diff,max_date,min_date
0,0,Muddy Run-Susquehanna River,Q1 2015,Q1,2015,962.805556,31.029108,268.833333,318.0,228.0,91,2019-07-10,2015-05-09
1,1,Muddy Run-Susquehanna River,Q1 2016,Q1,2016,208.883056,14.452787,220.883333,244.0,193.6,91,2019-07-10,2015-05-09
2,2,Muddy Run-Susquehanna River,Q1 2017,Q1,2017,1060.148760,32.559926,240.181818,291.0,179.0,91,2019-07-10,2015-05-09
3,3,Muddy Run-Susquehanna River,Q1 2018,Q1,2018,1606.355047,40.079359,234.613043,295.0,158.1,91,2019-07-10,2015-05-09
4,4,Muddy Run-Susquehanna River,Q1 2019,Q1,2019,777.696400,27.887209,222.140000,275.0,185.0,91,2019-07-10,2015-05-09


In [5]:
# check each df for huc names, if a huc appears in one df but not another, drop that huc from the source df
# sort hucs
all_hucs = []
suitable_hucs = []
bad_hucs = []

for df in df_list:
    hucs = df['huc'].unique()
    for huc in hucs:
        all_hucs.append(huc)
for huc in all_hucs:
    if all_hucs.count(huc) == 3:
        if huc not in suitable_hucs:
            suitable_hucs.append(huc)
    elif all_hucs.count(huc) < 3:
        if huc not in bad_hucs:
            bad_hucs.append(huc)

for df in df_list:
    df = df[df['huc'].isin(suitable_hucs)]

In [6]:
print(f"Found {len(suitable_hucs)} suitable hucs, and {len(bad_hucs)} un-usable hucs")

Found 29 suitable hucs, and 17 un-usable hucs


In [7]:
result_list = []
index = 0
# for each measure
for df in df_list:
    avg_over_time_list = []
     # for each suitable huc
    for huc in suitable_hucs: 
        # for each quarter
        huc_df = df.loc[df['huc'] == huc]
        for quarter in quarter_list:
            # find avg change over time
            working_df = huc_df.loc[df['q'] == quarter]
            earliest_avg = working_df['avg'].loc[working_df['year']==working_df['year'].min()].values[0]
            final_avg = working_df['avg'].loc[working_df['year']==working_df['year'].max()].values[0]
            percent_change = (final_avg - earliest_avg)/earliest_avg
            lowest_avg = working_df['avg'].min()
            highest_avg = working_df['avg'].max()
            q_avg_result = {
                'huc': huc,
                'quarter': quarter,
                'percent_change': percent_change,
                'highest_avg': highest_avg,
                'lowest_avg': lowest_avg,
                'start': working_df['year'].min(),
                'end': working_df['year'].max()
            }
            avg_over_time_list.append(q_avg_result)
    result_df = pd.DataFrame(avg_over_time_list)
    result_df.to_csv(f"../SourceMaterial/Measures/Results/Final_Analytics/{name_list[index]}_percent_change.csv")
    result_list.append(result_df)
    index+=1

In [8]:
result_list[0]

,huc,quarter,percent_change,highest_avg,lowest_avg,start,end
0,Upham Brook,Q1,-0.334152,18.500000,12.318182,2011,2020
1,Lower Nanticoke Creek,Q1,-0.527685,17.114286,7.041667,2012,2019
2,Patterson Creek-Susquehanna River,Q1,-0.356932,14.125000,8.166667,2014,2019
3,Upper Canacadea Creek,Q1,-0.848281,10.545833,1.600000,2013,2017
4,Karr Valley Creek,Q1,-0.906714,11.791667,1.100000,2013,2017
5,McHenry Valley Creek,Q1,-0.887365,11.541667,1.300000,2013,2017
6,Upper Cayuta Creek,Q1,-1.020599,12.950000,-0.250000,2011,2020
7,Headwaters Cayuta Creek,Q1,-0.965385,14.444444,0.500000,2011,2020
8,Sixmile Run,Q1,-0.311635,13.418803,8.899471,2011,2014
9,Laurel Run-Bald Eagle Creek,Q1,-0.451033,10.423611,5.722222,2013,2020


In [9]:
result_list[1]

,huc,quarter,percent_change,highest_avg,lowest_avg,start,end
0,Upham Brook,Q1,-0.102073,596.837502,176.800000,2011,2020
1,Lower Nanticoke Creek,Q1,-0.146900,276.750000,191.083333,2012,2019
2,Patterson Creek-Susquehanna River,Q1,-0.203728,342.952174,234.839583,2012,2019
3,Upper Canacadea Creek,Q1,0.104149,762.909091,487.932847,2012,2017
4,Karr Valley Creek,Q1,-0.563523,374.666667,163.533333,2013,2017
5,McHenry Valley Creek,Q1,-0.380560,312.090909,188.000000,2013,2017
6,Upper Cayuta Creek,Q1,0.130534,138.055556,93.982143,2011,2020
7,Headwaters Cayuta Creek,Q1,-0.086050,250.928571,171.000000,2011,2020
8,Sixmile Run,Q1,0.236992,70.842105,53.307692,2011,2014
9,Laurel Run-Bald Eagle Creek,Q1,-0.293747,122.500000,69.500000,2013,2020


In [10]:
result_list[2]

,huc,quarter,percent_change,highest_avg,lowest_avg,start,end
0,Upham Brook,Q1,0.102695,7.570000,6.722500,2011,2020
1,Lower Nanticoke Creek,Q1,-0.056211,7.571429,6.979167,2012,2019
2,Patterson Creek-Susquehanna River,Q1,0.074074,7.600000,6.750000,2013,2019
3,Upper Canacadea Creek,Q1,-0.090909,8.500000,7.500000,2013,2017
4,Karr Valley Creek,Q1,-0.095477,8.318182,7.500000,2013,2017
5,McHenry Valley Creek,Q1,-0.097744,8.363636,7.500000,2013,2017
6,Upper Cayuta Creek,Q1,-0.016571,7.054400,6.764583,2011,2020
7,Headwaters Cayuta Creek,Q1,-0.004739,7.297619,7.000000,2011,2020
8,Sixmile Run,Q1,-0.156863,6.000000,5.058824,2011,2014
9,Laurel Run-Bald Eagle Creek,Q1,-0.090909,5.800000,5.000000,2013,2020
